In [4]:
# Setup the Jupyter version of Dash
from jupyter_plotly_dash import JupyterDash #may need to put in comment/erase if using Dash to populate geomap

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#import CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# update with CRUD Python module & database (login was ready in CRUD python)

shelter = AnimalShelter()

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)  #initialize JupyterDash, may need to put in comment/erase to run Dash app
#app = dash. Dash(__name__)  #Need to uncomment to run dash app

#Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#Place the HTML image tag in the line below into the app.layout code according to your design
#Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    #Header
    html.A([
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
               height = 220, width = 220))], 
               href = 'https://www.snhu.edu'),
    #Unique identifier
    html.Center(html.B(html.H1('Philip Trinh - CS-340 Dashboard'))),
    html.Hr(),
        
#Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Hr(),
    html.Div([
        #Dropdown menu
        dcc.Dropdown(
        id='dropdown_filter',
        options=[
            {'label':'Water Rescue', 'value': 'Water'},
            {'label':'Mountain or Wilderness Rescue', 'value': 'MountainWild'},
            {'label':'Disaster Rescue or Individual Tracking', 'value':'DisasterInd'},
            {'label':'Reset', 'value': 'Reset'},
        ],
        value='Reset'
        ),
        ]),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=False,# not allow user to edit table/data
        style_table={'height': '100%', 'overflowY': 'auto'}, # Adjust table height
        page_size=10, #set rows per page
        sort_mode="multi", #allow sorting multiple column at once
        sort_action="native", # Enable column sorting
        row_selectable="single", # Enable single-row selection
        selected_rows=[0], # Initialize selected row (first row)
        filter_action="native", # Enable column filtering
        page_action="native", # Enable pagination for data table, splitting data into multiple pages
        page_current=0, # Set initial page display to first page   
    ), 

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

    
@app.callback([Output('datatable-id','data'),
              Output('datatable-id','columns')],
              Input('dropdown_filter', 'value'))
def update_dashboard(value):
    #Add code to filter interactive data table with MongoDB queries
    if (value =='Water'):
        df = pd.DataFrame(list(shelter.read(
            { "breed": {"$in": ["Chesapeake Bay Retriever", "Labrador Retriever Mix", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female", "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}})))
        
    elif (value == 'MountainWild'):
        df = pd.DataFrame(list(shelter.read(
            { "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}})))
    elif (value == 'DisasterInd'):
        df = pd.DataFrame(list(shelter.read(
            {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}})))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    
    return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # add code for chart of your choice (e.g. pie chart) 
    dffPie = pd.DataFrame.from_dict(viewData)

    return [
        dcc.Graph(            
            figure = px.pie(dffPie, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
       # Marker with tool tip and popup
       # Column 14 and 15 define the grid-coordinates for 
       # the map
       # Column 5 defines the breed for the animal
       # Column 10 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,14],dff.iloc[row,15]], children=[
                dl.Tooltip(dff.iloc[row,5]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,10])
                ])
            ])
        ])
    ]


#Run App
#may need to uncomment to run Dash app for geomap in event jupyter not able to display map via JupyterDash
#if __name__ == '__main__': app.run(host='0.0.0.0', port=10290, debug=True) 

app